# Workflow Summary
* Set up environment
* Load data
* Choose initial parameters
* Choose parameter bounds
* Define an objective function
    * artificial bounds
    * run forward model
    * apply sensor filter to modeled spectra
    * calculate closure/error
* Build objective function closure, binding specific values
* Minimisation step
* Run forward model on retrieved parameters
* Plot results

# Set up the environment:

In [ ]:
%matplotlib inline
import numpy as np
from scipy.optimize import minimize, basinhopping
import sambuca as sb
import matplotlib.pyplot as plt
from pkg_resources import resource_filename
from scipy.io import loadmat
import spectral as sp
import spectral.io.envi as envi

# set some controls on numpy formatting
# 5 decimal places, suppress scientific notation
np.set_printoptions(precision=5, suppress=True)

# set the matplotlib style to emulate ggplot2 from R
plt.style.use('ggplot')
plot_width = 12
plot_height = plot_width * 3/4

# spectra to plot
plot_items = list()

# Utility Functions

In [ ]:
def print_spectral_library(lib):
    #print(dir(lib))
    print("Named Bands:")
    for band_name in lib.names:
        print("\t" + band_name)
    print("Data dimensions: " + str(lib.spectra.shape))
    #print("Available Metadata:")
    #for key in lib.metadata:
    #    print("\t" + key)
        
        
def plot_spectra(plot_items, band_centers):
    plt.figure(figsize=(plot_width, plot_height))
    for pi in plot_items:
        plt.plot(band_centers, pi[0], label=pi[1])

    # set the X axis range
    plt.xlim(np.min(band_centers), np.max(band_centers))
            
    plt.legend(loc='upper right')
    plt.show()
    
    
def print_parameters(p):
    print('CHL:  {0:10.5f}\nCDOM: {1:10.5f}\nTR:   {2:10.5f}\nH:    {3:10.5f}\nQ:    {4:10.5f}'
          .format(p[0],p[1],p[2],p[4],p[3]))
    

def evaluate_forward_model(p, siops, sensor_filter):
    spectra = sb.forward_model(
        chl = p[0],
        cdom = p[1],
        nap = p[2],
        depth = p[4],
        substrate_fraction = p[3],
        substrate1=siops['substrate1'],
        substrate2=siops['substrate2'],
        wav=siops['wav'],
        awater=siops['awater'],
        aphy_star=siops['aphy_star'],
        num_bands=siops['d_wls'])
        
    if sensor_filter is not None:
        # Apply the sensor filter and return the results
        return sb.apply_sensor_filter(spectra.rrs, sensor_filter)
    else:
        return spectra.rrs

# Load Data
## Load the sambuca model inputs (SIOPS)

In [ ]:
filename = '../QBdata/forwardModelTestValues.mat'
siops_data = loadmat(filename, squeeze_me=True)
print(siops_data.keys())

filename = resource_filename(
            sb.__name__,
            'tests/data/test_resample.mat')
sensor_data = loadmat(filename, squeeze_me=True)
print()
print(sensor_data.keys())

print()
print(sensor_data['filt'].shape)

## Load the Quickbird data:

In [ ]:
%cd ../QBdata
%ls

In [ ]:
fullresspec = envi.open('FullResSpec.hdr','FullResSpec.lib')
print_spectral_library(fullresspec)

Grab the spectral data as a numpy array, and plot it to make sure it looks OK:

In [ ]:
fullres_spectra = fullresspec.spectra[0]
plot_items.clear()
plot_items.append((fullres_spectra, 'Full-res'))
plot_spectra(plot_items, fullresspec.bands.centers)

Load the Quickbird sensor filter:

In [ ]:
qbfilter = envi.open('QBFilter350_900nm.hdr','QBFilter350_900nm.lib')
print_spectral_library(qbfilter)

In [ ]:
plot_items.clear()
i = 0
for band in qbfilter.names:
    plot_items.append((qbfilter.spectra[i], band))
    i = i + 1
    
plot_spectra(plot_items,qbfilter.bands.centers)

Load the two target spectra, one generated with noise and one without:

In [ ]:
targetspec_nonoise = envi.open('QBSpecNoNoise.hdr','QBSpecNoNoise.lib')
print_spectral_library(targetspec_nonoise)

In [ ]:
targetspec_noise = envi.open('QBSpec0_001Noise.hdr','QBSpec0_001Noise.lib')
print_spectral_library(targetspec_noise)

In [ ]:
plot_items.clear()
plot_items.append((targetspec_nonoise.spectra[0], 'No Noise'))
plot_items.append((targetspec_noise.spectra[0], 'With Noise'))
plot_spectra(plot_items, targetspec_nonoise.bands.centers)

# Initial Parameters and Bounds

Load the parameter ranges from our test data

In [ ]:
filename = resource_filename(
            sb.__name__,
            'tests/data/test_optimise_data.mat')
data = loadmat(filename, squeeze_me=True)

p_min = data['p_min']
p_max = data['p_max']

#p_min and p_max have already been defined in previous cells (read from test data)
# random parameters often lead to a tougher optimisation challenge
p0_rand = np.random.random(5) * (p_max - p_min) + p_min

# Parameters known to lead Nelder-Mead astray:
#p0 = np.array([0.13959, 0.00361,  0.73770,  0.07821,  0.02446])

# Semi-reasonable first guess:
p0 = np.array([0.093, 0.013,  1.088,  0.815, 7.282])

# Very good first guess:
#p0 = np.array([0.145, 0.007,  0.25,  0.2, 7.45])

#p0 = p0_rand

print("Initial Parameters\n")
print_parameters(p0)

# repackage p_min and p_max into the tuple of (min,max) pairs expected by our objective function,
# and by the minimisation methods that support bounds
p_bounds = tuple(zip(p_min, p_max))
print('\nBounds = {}'.format(p_bounds))

#Objective Function
Define an objective builder that assembles sambuca components (forward model, sensor filter, error function), using a closure to bind the other inputs:

In [ ]:
def make_objective(siops, sensor_filter, obs_spectra, p_bounds=None, noise=None):
    awater = siops['awater']
    wav = siops['wav']
    aphy_star = siops['aphy_star']
    substrate1 = siops['substrate1']
    substrate2 = siops['substrate2']
    num_modelled_bands = sensor_filter.shape[1]
    num_observed_bands = sensor_filter.shape[0]
            
    def objective(p):
        # To support algorithms without support for boundary values, we assign a high 
        # score to out of range parameters.
        # p_bounds is a tuple of (min, max) pairs for each parameter in p
        if p_bounds is not None:
            for _p, lu in zip(p, p_bounds):
                l, u = lu
                if _p < l or _p > u:
                    return 100000.0
                    
        # call the forward model
        # We rely on the default values of the other model inputs, which happen to be
        # the same values used in the Matlab code
        results = sb.forward_model(
            chl = p[0],
            cdom = p[1],
            nap = p[2],
            depth = p[4],
            substrate_fraction = p[3],
            substrate1=substrate1,
            substrate2=substrate2,
            wav=wav,
            awater=awater,
            aphy_star=aphy_star,
            num_bands=num_modelled_bands)
        
        spectra = results.rrs
        
        # Apply the sensor filter
        filtered_spectra = sb.apply_sensor_filter(spectra, sensor_filter)
        
        # Calculate the error and return it as the objective score
        error = sb.error_all(obs_spectra, filtered_spectra, noise)
        
        #return error.alpha
        #return error.alpha_f
        return error.f
        #return error.lsq
    
    return objective

Define objective functions with noise and without:

In [ ]:
# for this test data, Steve used a constant noise of 0.001
noise = 0.001
#noise = None

sensor_filter = qbfilter.spectra

obj_bounded = make_objective(
        siops_data, 
        sensor_filter,
        targetspec_noise.spectra[0], 
        p_bounds,
        noise)

obj_unbounded = make_objective(
        siops_data, 
        sensor_filter,
        targetspec_nonoise.spectra[0], 
        p_bounds=None,
        noise=noise)

# Invert the model (parameter estimation)

## Nelder-Mead method:

In [ ]:
?sb.forward_model

In [ ]:
res_nm = minimize(obj_bounded, p0, method='nelder-mead', 
                  options={'xtol':1e-5, 'disp':True, 'maxiter':50000})
p_nm = res_nm['x']
print_parameters(p_nm)
nm_spectra = evaluate_forward_model(p_nm, siops_data, sensor_filter)
nm_spectra_full = evaluate_forward_model(p_nm, siops_data, None)

## Sequential Least-Squares

In [ ]:
res_slsqp = minimize(obj_unbounded, p0, method='SLSQP', bounds=p_bounds, 
                     options={'disp':True, 'maxiter':20000})
p_slsqp = res_slsqp['x']
print_parameters(p_slsqp)
slsqp_spectra = evaluate_forward_model(p_slsqp, siops_data, sensor_filter)
slsqp_spectra_full = evaluate_forward_model(p_slsqp, siops_data, None)

#Plot the results

In [ ]:
p0_spectra = evaluate_forward_model(p0, siops_data, sensor_filter)
p0_spectra_full = evaluate_forward_model(p0, siops_data, None)

## Filtered Spectra

In [ ]:
plot_items.clear()
#plot_items.append((targetspec_nonoise.spectra[0], 'Target'))
plot_items.append((targetspec_noise.spectra[0], 'Target'))
plot_items.append((nm_spectra, 'Nelder-Mead'))
plot_items.append((slsqp_spectra, 'Seq Least-Squares'))
plot_items.append((p0_spectra, 'p0'))
plot_spectra(plot_items, targetspec_noise.bands.centers)

##Full Spectra

In [ ]:
plot_items.clear()
#plot_items.append((targetspec_nonoise.spectra[0], 'Target'))
plot_items.append((fullres_spectra, 'Target'))
plot_items.append((nm_spectra_full, 'Nelder-Mead'))
plot_items.append((slsqp_spectra_full, 'Seq Least-Squares'))
plot_items.append((p0_spectra_full, 'p0'))
plot_spectra(plot_items, fullresspec.bands.centers)